In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

In [5]:
#df = pd.read_csv('data/liwc_raw_scores_full.csv') 
df = pd.read_csv('data/liwc_categorical_scores_full.csv') 

X = df.drop(['party'], axis=1)
y = df['party']


In [6]:
df_dem = df.loc[df['party'] == 'democrats']
df_rep = df.loc[df['party'] == 'republicans']


dem_mean = pd.DataFrame({'features': df_dem.mean().index, 'avg': df_dem.mean().values})
rep_mean = pd.DataFrame({'features': df_rep.mean().index, 'avg': df_rep.mean().values})

dem_mean['avg'] = dem_mean['avg']
rep_mean['avg'] = rep_mean['avg']

dem_mean.sort_values(by = ['avg'], ascending=False, inplace=True)
rep_mean.sort_values(by = ['avg'], ascending=False, inplace=True)


df_merged = dem_mean.merge(rep_mean, on='features')

df_merged['diff'] = df_merged['avg_x'] - df_merged['avg_y']
df_merged['diff'] = df_merged['diff'].abs()
df_merged.sort_values(by = ['diff'], ascending=False, inplace=True)
df_merged
#df_combo = pd.DataFrame(dem_mean['features'])

,features,avg_x,avg_y,diff
10,cogproc,0.069337,0.055145,0.014193
0,function,0.403427,0.389612,0.013815
2,relativ,0.131866,0.141712,0.009846
4,verb,0.112640,0.102873,0.009767
7,social,0.094723,0.086987,0.007735
15,work,0.055211,0.062878,0.007667
14,auxverb,0.056543,0.049008,0.007535
19,posemo,0.039836,0.047024,0.007188
11,space,0.069213,0.076143,0.006929
27,negemo,0.019961,0.013069,0.006891


In [ ]:
sorted_feature = np.argsort(dem_mean['avg'])
pos = np.arange(sorted_feature.shape[0]) + 0.5
plt.barh(pos, dem_mean['avg'], align='center', color='darkgreen')
#plt.barh(dem_mean['avg'], dem_mean['features'], align='center', color='darkgreen')
plt.title('Feature Importance')
plt.xlabel('Relative Feature Importance')
plt.yticks(pos, X.columns[sorted_feature])
fig = plt.gcf()
fig.set_size_inches(10,20)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=1, stratify=y)

label_encoder = LabelEncoder()
label_encoder.fit(y_train)

y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

y_train_category = to_categorical(y_train_encoded)
y_test_category = to_categorical(y_test_encoded)


In [ ]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(units=100,activation='relu', input_dim=X.shape[1]))
model.add(Dense(units=100,activation='relu'))
model.add(Dense(units=2, activation='softmax'))
          

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

result = model.fit(X_train, y_train_category,epochs=100, shuffle=True,verbose=2)

In [ ]:
model_loss, model_accuracy = model.evaluate(
    X_test, y_test_category, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
print(result.history['acc'])
print(result.history['loss'])

In [ ]:
import json

exDict = {'model_loss': model_loss, 'model_accuracy' : model_accuracy, 
          'acc':result.history['acc'], 'loss' :result.history['loss']}

with open('data/matrix/file.txt', 'w') as file:
     file.write(json.dumps(exDict))

In [ ]:
d2 = json.load(open("data/matrix/file.txt"))


In [ ]:
from keras.models import load_model
model = load_model("models/raw_full.h5")
#test = np.expand_dims(X_train[0], axis=0)

y_prob = model.predict(X_train[:1])
model.predict_classes(X_train[:1])
print(f"Predicted class: {model.predict_classes(X_train[5:6])}")
y_classes = y_prob.argmax(axis=-1)
y_prob
model.

In [ ]:
from keras.models import load_model
model = load_model("models/raw_full.h5")



In [ ]:
X_train[:6]


In [ ]:
X_train.iloc[0].shape

In [ ]:
predictions = model.predict_classes(X_test[:10])
prediction_labels = label_encoder.inverse_transform(predictions)
prediction_labels

print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:10])}")

In [ ]:
result = model.fit(X_train, y_train_category,epochs=100, shuffle=True,verbose=2)
#history = model.fit(X, Y, validation_split=0.33, nb_epoch=150, batch_size=10, verbose=0)
print(result.history['acc'])
print(result.history['loss'])

#print(result.history.keys())


In [ ]:

rf = RandomForestClassifier()
rf.fit(X,y)

imp_feature = rf.feature_importances_
sorted_feature = np.argsort(imp_feature)
print(imp_feature[sorted_feature])
pos = np.arange(sorted_feature.shape[0]) + 0.5
print(pos)
plt.barh(pos, imp_feature[sorted_feature], align='center', color='darkgreen')
plt.title('Feature Importance')
plt.xlabel('Relative Feature Importance')
plt.yticks(pos, X.columns[sorted_feature])
fig = plt.gcf()
fig.set_size_inches(10,20)

In [ ]:
import matplotlib.pyplot as plt

history = model.fit(x, y, validation_split=0.25, epochs=50, batch_size=16, verbose=1)

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()



In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')